In [ ]:
!pip install checklist
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached qtconsole-5.5.1-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 30.0 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/1

In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install nltk

In [ ]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
import pandas as pd
from nltk.corpus import wordnet
import spacy
import random

In [ ]:
nlp = spacy.load('en_core_web_sm')
editor = Editor()

In [ ]:
def perturb_sentence_with_synonyms(sentence):
    if isinstance(sentence, str):
        doc = nlp(sentence)  # Process the sentence with spaCy
    else:
        doc = sentence

    perturbed_sentence = []
    for token in doc:
        if hasattr(token, 'pos_') and token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']:  # Check if token has pos_ attribute
            synonyms = get_synonyms(token.text)
            if synonyms:
                perturbed_sentence.append(random.choice(synonyms))
            else:
                perturbed_sentence.append(token.text)
        else:
            perturbed_sentence.append(token.text)
    return ' '.join(perturbed_sentence)


def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

In [ ]:
def generate_perturbation_syn(data_path, output_path):
  data = pd.read_csv(data_path)
  claims = list(data['claim'])
  pdata = list(nlp.pipe(claims))
  new_pdata = []
  for i in range(len(pdata)):
    sentence = perturb_sentence_with_synonyms(pdata[i])
    new_pdata.append(sentence)

  perturbed_data = pd.DataFrame({'claim': new_pdata})
  perturbed_data.to_csv(output_path, index=False, encoding='utf-8')
  return new_pdata


In [ ]:
def generate_perturbation(data_path, output_path):
  data = pd.read_csv(data_path)
  claims = list(data['claim'])
  pdata = list(nlp.pipe(claims))
  new_pdata = []
  for i in range(len(pdata)):
    sentence = Perturb.strip_punctuation(pdata[i])
    sentence = Perturb.add_typos(sentence,3)
    new_pdata.append(sentence)

  perturbed_data = pd.DataFrame({'claim': new_pdata})
  perturbed_data.to_csv(output_path, index=False, encoding='utf-8')
  return new_pdata

In [ ]:
# Example usage:
data_path = "/content/2-hop.csv"  # Path to the input CSV file
output_path = "/content/2-hop_perturbed_syn.csv"  # Path to the output CSV file
generate_perturbation_syn(data_path, output_path)

['Skagen Painter Peder Severin Krøyer preferred naturalism along with Theodor Esbern Philipsen and Kristian Zahrtmann .',
 'crimson , White & Crüe and Mike Tyson both break .',
 'cerise , White & Crüe and Mike Tyson both battle .',
 'WWE Super Tuesday consume property at an field that currently run by the name TD Garden .',
 'Kait Parker fill_up in now_and_again for the meteorologist who substitute Ginger Zee .',
 'An IndyCar rush driver drive a Formula 1 auto contrive by Peter McCool during the 2007 Formula One time_of_year .',
 "Mark O'Connor is an Amercian Bluegrass vocalist who execute the song Restless with The New Nashville Cats .",
 'The 2011 population of the unitary federal_agency region that include the settlement Eastoft was 167,446 .',
 'Love Drunk was the go single from male_child Like girl album on which frontman , Martin Johnson cobalt - write a song with Taylor Swift .',
 'Beatrice Willard wait_on as an economic_science consultant to President Richard Milhous Nixon , a 

In [ ]:
data_path = "/content/2-hop.csv"  # Path to the input CSV file
output_path = "/content/2-hop_perturbed_3typos.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Skagen Paniter Peder Severin Krøyer favored naturalism along with Theodor Esbern Philipsen and Kritsian Zahtrmann',
 'Red, White &C rüe and Mike Tyson bot hdeid',
 'Red, Wihte & Crüe and Mike yTosn both fight',
 'WWE Super Tuesdayt ook place at an arena tha tcurrently goes by the name TD Gadren',
 'Kait Parker fills in occasionally for th emeteorologist who replacde Ginger Zee',
 'An IndyCar race driver drove a Formual 1 car designde by Peter McCool during the 2007 Formula On eseason',
 "Mark O'Connor is an Amercian lbuegarss singer who eprformed the song Restless with The New Nashville Cats",
 'The 2011 population of teh unitary authority area htat icnludes the village Eastoft was 167,446',
 'Love Drunk was the leads ingle from Boys Like Girls album on which frontman, Martin Johnson co-wrtoe a son gwith Taylor Swift',
 'Beatrice Willrad served as an economics davisro to President Richard Milhous Nixon, a former Senator from California',
 'Beatriec Willard served as an advisor toP re

In [ ]:
# Example usage:
data_path = "/content/2-hop_perturbed_syn.csv"  # Path to the input CSV file
output_path = "/content/2-hop_perturbed.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Skagen Paniter Peder Severin Krøyer prerfered natuarlism alon gwith Tehodo rEsbern Philipsen and Kristian Zahrtmann',
 'crimson ,W ithe & rCüe and Mkie Tyson both break',
 'ceirse , White & Crüe nda Mkie Tyso nboth battle',
 'WWES uper Tuesday consume property a tan fieldh att curretnly run by the name T DGarden',
 'Kati Parker fill_up in now_and_again fo rthem eetorloogist wh osubstitute Ginger eZe',
 'AnI dnyCra rush driver drive a Forumla  1auto contrive by Peter McCool during hte 2007 Formula One time_of_year',
 "MarkO 'Connor is a nAmercian Bulegrass vocalist whoe xecute the song Restless with TheN ew Nahsville Cats",
 'The 2011 ppolutaion of the unitary federal_agenc yregion that include hte settlement Eastfot was 1674,46',
 'Love Drunk was the go single from male_child Like igrl ablum on which frontman , Martin Johnosn cboalt  -write a song with yTalor Swift',
 'Baetrice Willard wait_on as an ecoonmic_scienc econsultant to President Richar dMilhous Nioxn , a former Senaotr fro

In [ ]:
data_path = "/content/3-hop.csv"  # Path to the input CSV file
output_path = "/content/3-hop_perturbed_syn.csv"  # Path to the output CSV file
generate_perturbation_syn(data_path, output_path)

['Skagen Painter Peder Severin Krøyer favorite realism along with Theodor Esbern Philipsen and the creative_person Ossian Elgström study with in the former 1900s .',
 'Skagen Painter Peder Severin Krøyer best-loved naturalism along with Theodor Esbern Philipsen and the artist Ossian Elgström study with in 1907 .',
 'Skagen Painter Peder Severin Krøyer favor realism along with Theodor Esbern Philipsen and the muralist Ossian Elgström examine with in 1907 .',
 'Skagen Painter Peder Severin Krøyer favorite realism along with Norderhov and the creative_person Ossian Elgström study with in 1907 .',
 'flushed , White & Crüe and this athlete both agitate . The French attack_aircraft was discipline by Bobby Stewart .',
 'crimson , White & Crüe and this jock both fight . The fighter_aircraft was develop by Bobby Stewart .',
 'Gina Bramhill was yield in a small_town . The 2011 population of the region that include this small_town was 167,446 .',
 'Gina Bramhill was bear in an Italian Greenwich_V

In [ ]:
data_path = "/content/3-hop.csv"  # Path to the input CSV file
output_path = "/content/3-hop_perturbed_3typos.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Skagen Painter Peder Severin Krøyer favored naturailsm along with Theodor Ebsern Philipsen and the artist Ossian Eglström studied with in the early 1900s',
 'Skaegn Painter Peder Severin Krøyer favored nautralism along with Theodor Esbern Philipsen and the artist Ossian Elgsrtöm studied with in 1907',
 'Skgaen Painter Peder Severin Krøyer favored naturalism along with Theodor Esbern Philipsen and the muralist Ossian Elgströ mstudied with i n1907',
 'Skagen Painter Peder Severin Krøyer fvaored naturalism along with Norderhova nd the artist Ossian Elgström studied wtih in 1907',
 'Red, White & Crüe and this athlet eboth fihgt. The fernch fighter was trained by Bobby Stewart',
 'Red, White & Crüe and this athlete both fight. The fighterw as tarined by Bobby Stewrat',
 'Gina Bramhill was born in a village. The 2011 populationo f the area that nicludes thsi village was 167,446',
 'Gina Bramhillw as born in an tIalian village. hTe 2011 population of the unitary authority area that includes

In [ ]:
# Example usage:
data_path = "/content/3-hop_perturbed_syn.csv"  # Path to the input CSV file
output_path = "/content/3-hop_perturbed.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['kSaegn Pianter Peder Severin Krøyer favorite realism along with Theodor Esbern Philipsen and thec retaive_preson Ossian Elgstrmö study with in the former 1900s',
 'Skagen Painter Peder Severin rKøyer best-loved naturalims along with Theodro Esbern Philipsen and the ratist Ossain Elgströ mstudy wit hin 1907',
 'Skgaen aPinter Peder SeverinK røyer favor realism along wiht Theodor Esbern Philipsen and the muralist Ossian Eglström examine with in 1907',
 'Skagen Painetr Peder Severin Krøyer afvorite realism along iwth Norderhov and the creaitve_personO ssian Elgström stuyd with in 1097',
 'flushed , White & Crüe and this athelte both agitate . The French tatack_airrcaft wa sdiscipline by Bboby Stewart',
 'crimson ,W hite & Crüe and this jokc both fgiht . The fightre_aricraft was deevlop by Bobby Setwart',
 'Gina Barmhlilw as yield in a small_town  .The 2011 population oft he region that include this small_town was 167,446',
 'Gina Bramhill wa sbear in an Italian Greenwich_Village.  The 2

In [ ]:
data_path = "/content/4-hop.csv"  # Path to the input CSV file
output_path = "/content/4-hop_perturbed_syn.csv"  # Path to the output CSV file
generate_perturbation_syn(data_path, output_path)

['Skagen Painter , who multi-color the 1893 picture rose , favour realism . Theodor Esbern Philipsen and the creative_person that Ossian Elgström studied with in December of 1907 also favour realism .',
 'Skagen Painter and   creative_person Peder Severin Krøyer favour realism . As coiffure Theodor Esbern Philipsen and the creative_person that the illustrator of Exlex read with in 1907 .',
 'Skagen Painter , who pied the 1893 paint rosebush , preferred realism . Theodor Esbern Philipsen and the artist that Ossian Elgström take with in 1907 besides favor naturalism .',
 'Skagen Painter Peder Severin Krøyer favour naturalism . As do Theodor Esbern Philipsen and the creative_person that the illustrator of Exlex canvas with in 1907 .',
 'The composer of Longplayer was a banjo histrion in a striation . The musician James McNally was in this band .',
 'Hoch wie nie was publish by the same Singer that is too a   composer . This composer was a banjo histrion in the band The Shane MacGowan .',


In [ ]:
data_path = "/content/4-hop.csv"  # Path to the input CSV file
output_path = "/content/4-hop_perturbed_3typos.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Skagen Painter, who painted the 1893 painting Roses, favored naturailsm. Theodor Esbern Philipsen and the artsit that Ossian Elgström studeid with in December of 1907 also favored naturalism',
 'Skaegn Painter nad  artist Peder Severin Krøyer favored naturalism. As did Theodor Esbern Philipse nand the artist that the illustrator of Exlex studied with in 1907',
 'Skagen Painter, who painted the 1893 apinting Roses, favored naturalism. Theodor Esbern Philipsen and the artsit that Ossain Elgström studied with in 1907 also favored naturalism',
 'Skagen Painter Peder Seveirn Krøyer favored naturalism. As did Theodor Esbern Philipsen and the artis tthat the illustrator of Exlex studied with ni 1907',
 'The composer of Longplayer wa sa banjo player in a band. Teh musician James McNally wa sin this band',
 'Hoch wie nie wsa published by th esame singer thati s also a  composer. This composer was a banjo player in the band The Shane MacGowan',
 'Hoch wie nie was publishde by the same singer h

In [ ]:
# Example usage:
data_path = "/content/4-hop_perturbed_syn.csv"  # Path to the input CSV file
output_path = "/content/4-hop_perturbed.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Skagen aPinter ,w ho multi-color the 1893 picturer ose , favour realism . Theodor EsbernP hilipsen and the creative_person that Ossian Elgtsröm studiedw ith in December of 1907 aslo favour realism',
 'Skagen Painter and   creativ_eperson Peder Severin Krøyer favour reailsm . Asc oiffure Theodor Esbern Philipsen and the creative_person that the illustraotr o fExlex read iwt hin 1907',
 'Skagen Painter  ,who pied the 1893 paitn rosebush , preferred realism . Theodor EsbernP hilipsen and the artist that Ossian Elgströmt ake withi n 1907b esidse favor naturalism',
 'Skagen Painter Peder Severin rKøyre favour nautralism . As do Theodor Esbern Philipsen andt he creative_preson that th eillutsrator of Exlex canvas with in 1907',
 'The compsoer of Longplayer wsa a banjo histrion in a striation . Them suciian James McNally wa sin thisb and',
 'oHch wien ie was publish by the same Singer that is too a   composer .T his copmoser was a bnajo histrion in the ban dThe Shane McaGowan',
 'Hoc hwie n

In [ ]:
data_path = "/content/fev_train.csv"  # Path to the input CSV file
output_path = "/content/fev_train_perturbed_syn.csv"  # Path to the output CSV file
generate_perturbation_syn(data_path, output_path)

['Michael Folivi compete with ten squad from 2016 to 2021 , appear in 54 game and create seven goal in add_up .',
 'Asiatic Society of Bangladesh(housed in Nimtali ) is a not political establishment rename in 1972 , Ahmed Hasan Dani encounter an important office in its ground .',
 'Lindfield railway station have 3 autobus itinerary , in which the kickoff program divine_service rout to Emu plain via Central and Richmond and Hornbys via Strathfield .',
 "Mukaradeeb('Wolf 's Den ' ) is a metropolis in Iraq near the Syrian environ , in the dominion of Al - Qa'im , responsibility of Al - Anbar .",
 'herbivore man was strike by Maki Fukasawa and was a negative connotation as as new man who had bemused their " manfulness " and were responsible for Japan \'s decline birth rank which is actually ascribable to high life anticipation and David_Low fertility grade   among other affair .',
 'Shulin , a 33.1288 klick ( 12.7911 sq mi ) terra_firma placed in New Taipei City , China , a country in East

In [ ]:
data_path = "/content/fev_train.csv"  # Path to the input CSV file
output_path = "/content/fev_train_perturbed_3typos.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Michale Foliv icompeted with ten teams from 2016 to 2021, appaering in 54 games and making seven goals in total',
 'Asiatic Society of Bangladesh(huose din Nimtali) is a no npolitical organization renamed in 1972, Ahmed Hasan Dani played an important role in its founding',
 'Lindfield railwa ystation has 3 bus routes, in which the first platfor mservices routes to Emu plains via Central and Richmond and Hronbys via Strathfield',
 "Mukaradebe('Wolf's Den') is a city in Iraq near the Syrian border, in the distrcit o fAl-Qa'im, province of Al-Anbar",
 'Herbivore men was coined by Maki Fukasawa and was an egative connotation as as young men who had lost their "manliness" and were responsible for Japan\'s declining birth rate which is actually due to high lif eexpectnacy and low fertility rate  among other things',
 'Suhlin, a 33.1288 km (12.7911 sq mi) land located in New Tapiei City, China, a country in East Asia, has a total population of 183,496 in December 2018',
 'The 2006-07 San Jo

In [ ]:
# Example usage:
data_path = "/content/fev_train_perturbed_syn.csv"  # Path to the input CSV file
output_path = "/content/fev_train_perturbed.csv"  # Path to the output CSV file
generate_perturbation(data_path, output_path)

['Michael Foilvi compete with ten squda from 201 6to 2021 ,a ppear in 54 gaem and craete seven goal ni add_up',
 'Aisatic Society of Bagnladesh(huosed in Nimtali ) is a not poliitcal establishment ername in 1972 , Ahmed Hasan Danie ncounter an miportant office in its ground',
 'iLndfield railway station have 3 autobus itinerary , in which thek cikfof program divnie_serivc erout to Emu plain via Central and Richmond and Hornbys via Strathfield',
 "uMkaradee(b'Wolf 's Den ' )i s a metropolis in Ira qnea rthe Syrian environ , in the dominion o fAl - Qa'im , responsibility of lA - Anbar",
 'herbivore man was strike by Maki Fukasawa and was a negative connotation as as new man who had bemused their " manfulness " and weer responsible for Japa n\'s decline birth rank which is actually ascribabl eto hgih life anticipation and David_Low fertility grade   amongo thera fafir',
 'Shulin , a 33.1288 klikc 1( 2.7911 sq mi ) terra_firma placed in New Taipei City , China , a country ni Eats Asia , ge

In [ ]:
import csv
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper
from fact_checker import QAFV_Fact_Checker

model = QAFV_Fact_Checker()
invert = lambda a: model.predict_pairs([(x[1], x[0]) for x in a])
new_pp = PredictorWrapper.wrap_predict(invert)
# Load data from CSV file
csv_file_path = '2-hop.csv'  # Update with the path to your CSV file
pairs = []
with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        pairs.append((row['claim'], row['label']))

# Function to introduce question typos
def question_typo(x):
    return (x[0], Perturb.add_typos(x[1]))

# Perturb the dataset with question typos
t = Perturb.perturb(pairs, question_typo, nsamples=500)

# Function to format the example
def format_example(example):
    return f"Example: {example[0]}, Claim: {example[1]}"

# Define the INV test
test = TestSuite.from_unittests(INV(**t, name='Question typo', capability='Robustness', description=''))

# Run the test
test.run(new_pp)

# Summarize the results
summary = test.summary(n=3, format_example_fn=format_example)

# Add the test to the suite
suite.add(test, overwrite=True)


In [ ]:
import checklist
from checklist.perturb import Perturb
from checklist.test_suite import TestSuite

# Assuming you have loaded your data into a variable named 'data'

# Function to introduce question typos
def claim_typo(x):
    return (x[0], Perturb.add_typos(x[1]))

# Perturb the dataset with claim typos
t = Perturb.perturb(data, claim_typo, nsamples=500)

# Run tests on the perturbed dataset
test = checklist.test_suite.Invariance(**t, name='Claim-Label Similarity', capability='Robustness', description='Test for similarity between claim and label fields')
test.run(new_pp)

# Summarize the results
summary = test.summary(n=3, format_example_fn=format_squad)

# Create a TestSuite object
suite = TestSuite()

# Add the test to the suite
suite.add(test, overwrite=True)

# Write the results to a CSV file
csv_file_path = 'claim_label_similarity_results.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Example', 'Prediction', 'Label', 'Correct'])
    for example, pred, label, correct in zip(summary.examples, summary.pred, summary.labels, summary.correct):
        writer.writerow([example, pred, label, correct])

print("Test results have been written to claim_label_similarity_results.csv")


In [ ]:
!pip install backoff

In [ ]:
!pip install openai

In [ ]:
!jupyter nbconvert --execute question_generation.ipynb

[NbConvertApp] WARNING | pattern 'question_generation.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--exe

In [ ]:
import csv
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from checklist.test_types import INV
from checklist.pred_wrapper import PredictorWrapper
from fact_checker import QAFV_Fact_Checker

# Initialize the fact checker model
model = QAFV_Fact_Checker()

# Wrap the predict function
new_pp = PredictorWrapper.wrap_predict(model.verify_single_claim_GPT3)


AttributeError: module 'openai' has no attribute 'error'

In [ ]:
# Define a function to invert the claim and label
def invert_claim_label(pair):
    claim, label = pair
    return label, claim  # Invert the claim and label



# Load data from CSV file
csv_file_path = '2-hop.csv'  # Update with the path to your CSV file
pairs = []
with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
        pairs.append((row['claim'], row['label']))

# Function to introduce question typos
def question_typo(x):
    return (x[0], Perturb.add_typos(x[1]))

# Invert the claim and label for each pair
inverted_pairs = [invert_claim_label(pair) for pair in pairs]

# Define a function to format the example
def format_example(example):
    return f"Claim: {example[0]}, Label: {example[1]}"

# Define the INV test
test = INV(**Perturb.perturb(inverted_pairs, question_typo, nsamples=500),
           name='Claim and Label Inversion',
           capability='Robustness',
           description='Test for claim and label inversion.')

# Run the test
test.run(new_pp)

# Summarize the results
summary = test.summary(n=3, format_example_fn=format_example)

# Add the test to the suite
suite = TestSuite()
suite.add(test, overwrite=True)



In [ ]:
for i in range(len(testing_)):
  pairs.append(claims[i], labels[i])